<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 9


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Описание задачи:

Создать базовый класс Order в C#, который будет представлять информацию о заказах товаров или услуг. На основе этого класса разработать 2-3 производных класса, демонстрирующих принципы наследования и полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и методы, а также переопределены некоторые методы базового класса для демонстрации полиморфизма.

Требования к базовому классу Order:

Атрибуты: ID заказа (OrderId), Дата создания (CreationDate), Сумма заказа (TotalAmount).
Методы:
CalculateTotal(): метод для расчета общей суммы заказа.

AddItem(Item item): метод для добавления элемента в заказ.

RemoveItem(Item item): метод для удаления элемента из заказа.

Требования к производным классам:

ОнлайнЗаказ (OnlineOrder): Должен содержать дополнительные атрибуты, такие как Email клиента (CustomerEmail). Метод AddItem() должен быть переопределен для добавления информации о способе доставки при добавлении элемента.
ФизическийЗаказ (PhysicalOrder): Должен содержать дополнительные атрибуты, такие как Адрес доставки (DeliveryAddress). Метод RemoveItem() должен быть переопределен для добавления информации о возврате товара при удалении элемента.
СпециализированныйЗаказ (SpecializedOrder) (если требуется третий класс): Должен содержать дополнительные атрибуты, такие как Специальные условия (SpecialConditions). Метод CalculateTotal() должен быть переопределен для учета специальных условий при расчете общей суммы заказа

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public class Order// обобщенный generic класс
{
    public int OrderId { get; set; } // поля класса
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    protected List<Item> Items { get; set; } // позволяет хранить упорядоченные элементы

    public Order(int orderId) // констр
    {
        OrderId = orderId;
        CreationDate = DateTime.Now;
        Items = new List<Item>();
        TotalAmount = 0;
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in Items)
        {
            TotalAmount += item.Price;
        }
    }

    public virtual void AddItem(Item item)
    {
        Items.Add(item);
        CalculateTotal();
    }

    public virtual void RemoveItem(Item item)
    {
        if (Items.Remove(item))
        {
            CalculateTotal();
        }
    }

    public virtual void PrintDetails()
    {
        Console.WriteLine($"Заказ ID: {OrderId}, Дата создания: {CreationDate}, Общая сумма: {TotalAmount} р.");
    }

    
    
}

public class Item // дополнительный класс
{
    public string Name { get; set; }
    public decimal Price { get; set; }
    public Item(string name, decimal price) // констр
    {
        Name = name;
        Price = price;
    }
    public virtual void GetInfo()
    {
        Console.WriteLine($"Товар: {Name}, Цена: {Price}");
    }
}

///////////////////////////////////////////////////////////// generic 
public class Order<T> where T : Item // Т - класс содержащий инфу о товаре (название цена)
{
    private List<T> allItems = new List<T>(); //список всех товаров в заказе
    public decimal TotalPrice { get; private set; }

    public void AddItem(T item)
    {
        allItems.Add(item);
        CalculateTotal();
    }

    public void RemoveItem(T item)
    {
        allItems.Remove(item);
        CalculateTotal();
    }

    private void CalculateTotal()
    {
        TotalPrice = 0;
        foreach (var item in allItems)
        {
            TotalPrice += item.Price;
        }
    }

    public void PrintDetails()
    {
        foreach (var item in allItems)
        {
            item.GetInfo();
        }
        Console.WriteLine($"Общая стоимость: {TotalPrice}");
    }

    public static Order<T> operator +(Order<T> order1, Order<T> order2) //перегрузка оператора позволяет объединять два заказа создавая новый
    {
        Order<T> Orders = new Order<T>();
        foreach (var item in order1.allItems)
        {
            Orders.AddItem(item);
        }
        foreach (var item in order2.allItems)
        {
            Orders.AddItem(item);
        }
        return Orders;
    }
}
///////////////////////////////////////////////////////////////////

public class OnlineOrder : Order// обобщенный класс
{
    public string CustomerEmail { get; set; }
    public OnlineOrder(int orderId, string customerEmail) : base(orderId)
    {
        CustomerEmail = customerEmail;
    }
    public override void AddItem(Item item)
    {
        base.AddItem(item);
        PrintAdd(item);
    }
    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        PrintRemove(item);
    }
    private void PrintAdd(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' добавлен в онлайн-заказ для {CustomerEmail}");
    }
    private void PrintRemove(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' удален из онлайн-заказа.");
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Email клиента: {CustomerEmail}");
    }
}

public class PhysicalOrder : OnlineOrder // сложное наследование
{
    public string DeliveryAddress { get; set; }
    public PhysicalOrder(int orderId, string deliveryAddress) : base(orderId, "---")
    {
        DeliveryAddress = deliveryAddress;
    }
    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Адрес доставки: {DeliveryAddress}");
    }
}

public interface ISpecializedOrder // интерфейс
{
    void SetSpecialConditions(string specialConditions);
    void Sale(Item item);
}

public class SpecializedOrder : Order, ISpecializedOrder //  множ наследование
{
    public string SpecialConditions { get; private set; }
    public bool HasSpecialConditions { get; private set; } // булева переменная
    public SpecializedOrder(int orderId, string specialConditions) : base(orderId)
    {
        SetSpecialConditions(specialConditions);
    }
    public void SetSpecialConditions(string specialConditions)
    {
        SpecialConditions = specialConditions;
        HasSpecialConditions = !string.IsNullOrEmpty(specialConditions); // проверка на пустоту или null
    }
    public void Sale(Item item)
    {
        Console.WriteLine($"На товар '{item.Name}' действует скидка 10%!");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (HasSpecialConditions) 
        {
            TotalAmount *= 0.9m; 
        }
    }

    public override void PrintDetails()
    {
        base.PrintDetails();
        if (HasSpecialConditions)
        {
            Console.WriteLine($"Специальные условия: {SpecialConditions}");
        }
        else
        {
            Console.WriteLine("Специальные условия отсутствуют.");
        }
    }
}




Item item1 = new Item("Книга", 150); 
Item item2 = new Item("Игрушка", 200);
Item item3 = new Item("Подушка", 700);

OnlineOrder onlineOrder = new OnlineOrder(1, "pochta@mail.ru");
onlineOrder.AddItem(item1);
onlineOrder.PrintDetails();
Console.WriteLine();

PhysicalOrder physicalOrder = new PhysicalOrder(2, "ул. Мельникайте, д. 70, г. Тюмень");
physicalOrder.AddItem(item2);
physicalOrder.PrintDetails();
Console.WriteLine();

SpecializedOrder specializedOrder = new SpecializedOrder(3, "Скидка на новый продукт");
specializedOrder.AddItem(item3);
specializedOrder.Sale(item3);
specializedOrder.CalculateTotal(); 
specializedOrder.PrintDetails();
Console.WriteLine();

Order<Item> order1 = new Order<Item>();
Order<Item> order2 = new Order<Item>();    
   
order1.AddItem(item1);
order1.AddItem(item2);
order2.AddItem(item3);

Order<Item> combinedOrder = order1 + order2;
Console.WriteLine("Детали обобщенного заказа:");
combinedOrder.PrintDetails();